In [1]:
import glob
from PIL import Image
import xmltodict
import os
import re
from collections import defaultdict

def read_xmls(folder_path):
    ''' 
    
    return a tuple
    
    (pic filename, list of objects) '''
    folder_path += '/*.xml'
    file_name = glob.glob(folder_path)
    file_use = []
    data = []
    for file in file_name:
        data_read = xmltodict.parse(open(file,'r').read())
        if 'object' in data_read['annotation']:
            data.append((file, data_read['annotation']['object']))
    return data

def get_obj_name(folder_path):
    '''
    arg:
        folder path
    return:
        all object_name form xml files in the folder
    '''
    folder_path += '/*.xml'
    file_name = glob.glob(folder_path)
    s_name = set()
    for file in file_name:
        data = xmltodict.parse(open(file,'r').read())
        if 'object' in data['annotation']:
            if isinstance(data['annotation']['object'],list):
                for item in data['annotation']['object']:
                    s_name.add(item['name'])
            else:
                s_name.add(data['annotation']['object']['name'])
    return s_name


get_obj_name('./dataFinal')

{'7days',
 'coke',
 'evian',
 'maltesers',
 'redbull',
 'skittles',
 'snickers',
 'twix'}

In [2]:
crop_pic = defaultdict(list)
data = read_xmls('./dataFinal')
len(data)

366

In [5]:
print('srart')
for i,each_pic in enumerate(data):
    if (i+1) % 36 == 0:
        print(f'progress: {((i//36)+1)*10}%')
    names = glob.glob(each_pic[0][:-3] + '**')
    for name in names:
        if not re.search('xml',name):
            pic_name = name        
    im = Image.open(pic_name)
    im_list = []
    if isinstance(each_pic[1],list):
        for obj in each_pic[1]:
            c_im = im.crop((int(obj['bndbox']['xmin']),
                       int(obj['bndbox']['ymin']),
                       int(obj['bndbox']['xmax']),
                       int(obj['bndbox']['ymax'])))
            crop_pic[obj['name']].append(c_im)
    else:
        c_im = im.crop((int(each_pic[1]['bndbox']['xmin']),
                       int(each_pic[1]['bndbox']['ymin']),
                       int(each_pic[1]['bndbox']['xmax']),
                       int(each_pic[1]['bndbox']['ymax'])))
        crop_pic[each_pic[1]['name']].append(c_im)

srart
progress: 10%
progress: 20%
progress: 30%
progress: 40%
progress: 50%
progress: 60%
progress: 70%
progress: 80%
progress: 90%
progress: 100%


In [4]:
count = 0
for item in crop_pic:
    count += len(crop_pic[item])
    
count

2203

In [5]:
progress = 1
print('start')
for item in (crop_pic):
    for i, im in enumerate(crop_pic[item]):
        if progress % 220 == 0:
            print(f'pregress: {(progress//220)*10}%')
        im.save(f'./cropPics/{item}{i}.png')
        progress += 1

start
pregress: 10%
pregress: 20%
pregress: 30%
pregress: 40%
pregress: 50%
pregress: 60%
pregress: 70%
pregress: 80%
pregress: 90%
pregress: 100%
